In [ ]:
import pandas as pd

# 1. Load both datasets
# Make sure the filenames and paths are correct
kaggle_file_path = r'C:\Users\HRITHIK S\MY PROJECTS\football-predictor\data\raw\english_premier_league_2000-2025.csv'
df_kaggle = pd.read_csv(kaggle_file_path)

api_file_path = r'C:\Users\HRITHIK S\MY PROJECTS\football-predictor\data\raw\premier_league_matches.csv'
df_api = pd.read_csv(api_file_path)


# 2. --- Clean and prepare the Kaggle Data ---
# Select only the columns we need
df_kaggle_clean = df_kaggle[['Season', 'MatchDate', 'HomeTeam', 'AwayTeam', 'FullTimeHomeGoals', 'FullTimeAwayGoals', 'FullTimeResult']].copy()

# Rename columns to match our API data's format
df_kaggle_clean.rename(columns={
    'Season': 'season',
    'MatchDate': 'date',
    'HomeTeam': 'home_team',
    'AwayTeam': 'away_team',
    'FullTimeHomeGoals': 'home_goals',
    'FullTimeAwayGoals': 'away_goals',
    'FullTimeResult': 'result_short' # Rename temporarily before mapping
}, inplace=True)

# Map the result ('H', 'A', 'D') to the same format as our API data
df_kaggle_clean['result'] = df_kaggle_clean['result_short'].map({'H': 'HOME_TEAM', 'A': 'AWAY_TEAM', 'D': 'DRAW'})
df_kaggle_clean.drop(columns=['result_short'], inplace=True)

# Convert the date column to datetime objects for proper sorting
df_kaggle_clean['date'] = pd.to_datetime(df_kaggle_clean['date'])

# Extract the starting year of the season (e.g., '2023/24' -> 2023)
df_kaggle_clean['season'] = df_kaggle_clean['season'].apply(lambda x: int(x.split('/')[0]))


# 3. Filter for the 7 seasons we want (2018-2025)
df_kaggle_clean = df_kaggle_clean[df_kaggle_clean['season'] >= 2018]


# 4. --- Combine the historical and recent data ---
df_combined = pd.concat([df_kaggle_clean, df_api], ignore_index=True)


# 5. --- Final Cleanup ---
# Sort by date and remove any duplicate matches, keeping the last one
df_combined.sort_values('date', inplace=True)
df_combined.drop_duplicates(subset=['date', 'home_team', 'away_team'], keep='last', inplace=True)


# --- Final Verification ---
print("Successfully combined historical (Kaggle) and recent (API) data.")
print(f"Total matches in the final dataset: {len(df_combined)}")
print("\nLast 5 rows of the new combined dataset:")
print(df_combined.tail())

# We will now use df_combined as our main DataFrame for the rest of the project
df = df_combined.copy()